# Notebook pour voir les résultats des DMPs : Pour GESRes_dataset

In [11]:
from video import Video
import visualize
from utils import landmarks_2_table
import mediapipe as mp
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from IPython.display import clear_output, display
import random



In [13]:
import numpy as np

class DMPVector:
    def __init__(self, n_basis: int = 20):
        self.n_basis = n_basis # number of gaussians
        self.centers = None
        self.widths = None
        self.W = None       # (n_basis, D)
        self.y0 = None      # (D,)
        self.g = None       # (D,)
        self.T_train = None

    def _basis_functions(self, T: int):
        t = np.linspace(0.0, 1.0, T)
        self.centers = np.linspace(0.0, 1.0, self.n_basis)
        self.widths = np.ones(self.n_basis) * (self.n_basis ** 2)
        psi = np.exp(-self.widths * (t[:, None] - self.centers[None, :])**2)
        return psi, t

    def fit(self, Y: np.ndarray):
        T, D = Y.shape
        self.T_train = T
        self.y0 = Y[0].copy()
        self.g  = Y[-1].copy()

        psi, t = self._basis_functions(T)   # (T, n_basis)
        Y_lin = self.y0 + np.outer(t, (self.g - self.y0))   # (T, D)
        F_target = Y - Y_lin                                # (T, D)

        psi_sum  = np.sum(psi, axis=1, keepdims=True) + 1e-10
        psi_norm = psi / psi_sum                           # (T, n_basis)

        self.W, _, _, _ = np.linalg.lstsq(psi_norm, F_target, rcond=None)

    def fit_multi(self, Y_batch: np.ndarray):
        N, T, D = Y_batch.shape
        self.T_train = T

        psi, t = self._basis_functions(T)           # (T, n_basis)
        psi_sum  = np.sum(psi, axis=1, keepdims=True) + 1e-10
        psi_norm = psi / psi_sum                    # (T, n_basis)

        F_list = []
        y0_all = []
        g_all  = []

        for i in range(N):
            Y = Y_batch[i]
            y0_i = Y[0].copy()
            g_i  = Y[-1].copy()

            y0_all.append(y0_i)
            g_all.append(g_i)

            Y_lin_i = y0_i + np.outer(t, (g_i - y0_i))   # (T, D)
            F_i     = Y - Y_lin_i                        # (T, D)
            F_list.append(F_i)

        F_all = np.concatenate(F_list, axis=0)           # (N*T, D)
        Psi_big = np.tile(psi_norm, (N, 1))              # (N*T, n_basis)

        self.W, _, _, _ = np.linalg.lstsq(Psi_big, F_all, rcond=None)

        self.y0 = np.mean(np.stack(y0_all, axis=0), axis=0)
        self.g  = np.mean(np.stack(g_all,  axis=0), axis=0)

    def rollout(self, T: int = None, amplitude_scale: float = 1.0) -> np.ndarray:
        if T is None:
            T = self.T_train

        psi, t = self._basis_functions(T)
        psi_sum  = np.sum(psi, axis=1, keepdims=True) + 1e-10
        psi_norm = psi / psi_sum

        F = psi_norm @ self.W * amplitude_scale
        Y_lin = self.y0 + np.outer(t, (self.g - self.y0))
        Y = Y_lin + F
        return Y


In [14]:
import joblib

gesture_dmps = joblib.load("gesture_dmps.joblib")
print(gesture_dmps.keys())


dict_keys(['one', 'finger', 'cup', 'knock', 'clasp', 'hold', 'punch', 'point', 'wave', 'show', 'soft', 'present', 'hand'])


In [32]:
KEYPOINTS = [2, 5, 9, 10, 11, 12] + list(range(13, 23))



In [16]:
# Convert 32-dim continuous vector to 16x2 skeleton

def vec32_to_skeleton(Y_vec: np.ndarray) -> np.ndarray:
    """
    Y_vec : (T, 32)
    -> (T, 16, 2)
    """
    T, D = Y_vec.shape
    assert D == 32, f"Expected 32 dims, got {D}"
    return Y_vec.reshape(T, 16, 2)


In [20]:
def rotate_skeleton_xy(seq: np.ndarray) -> np.ndarray:
    """
    Prend un tableau (T, J, 2) et échange les colonnes x et y.
    Retourne un nouveau tableau (ne modifie pas l'original).
    """
    rotated = seq.copy()
    pts = rotated.copy()
    rotated[:, :, 0] = pts[:, :, 1]  # nouveau x = ancien y
    rotated[:, :, 1] = pts[:, :, 0]  # nouveau y = ancien x
    return rotated



In [33]:
EDGES = [
    # Visage simplifié
    (0, 1),   # yeux
    (0, 2),   # oeil gauche -> coin bouche gauche
    (1, 3),   # oeil droit  -> coin bouche droit
    (2, 3),   # bouche

    # Épaules
    (4, 5),   # épaules

    # Bras gauche (on suppose : 4 = épaule gauche, 6 = coude gauche, 8 = poignet gauche)
    (4, 6),   # épaule -> coude
    (6, 8),   # coude  -> poignet
    (8, 10),  # poignet -> pinky
    (8, 12),  # poignet -> index
    (8, 14),  # poignet -> pouce

    # Bras droit (5 = épaule droite, 7 = coude droit, 9 = poignet droit)
    (5, 7),   # épaule -> coude
    (7, 9),   # coude  -> poignet
    (9, 11),  # poignet -> pinky
    (9, 13),  # poignet -> index
    (9, 15),  # poignet -> pouce
]

def plot_stickman_frame2(ax, norm_seq, frame_idx: int):
    pts = norm_seq[frame_idx]          # (33, 2)
    visible = ~np.isnan(pts).any(axis=1)

    ax.cla()   # efface le contenu des axes

    # segments
    for i, j in EDGES:
        if i < pts.shape[0] and j < pts.shape[0]:
            if visible[i] and visible[j]:
                x_vals = [pts[i, 0], pts[j, 0]]
                y_vals = [pts[i, 1], pts[j, 1]]
                ax.plot(x_vals, y_vals, "-")

    # points visibles
    x_vis = pts[visible, 0]
    y_vis = pts[visible, 1]
    ax.scatter(x_vis, y_vis, c="red")

    # indices
    for idx in np.where(visible)[0]:
        x, y = pts[idx]
        ax.text(x, y, str(idx), fontsize=7)

    ax.set_title(f"Stickman (frame {frame_idx})")
    ax.axis("equal")
    ax.grid(True)

In [34]:
T=50
def plot_stickman(label):
  fig, ax = plt.subplots(figsize=(6, 6))
  for i in range(0, min(T,50)):
    if label not in gesture_dmps:
        raise ValueError(f"Label '{label}' pas trouvé dans gesture_dmps.")

    dmp = gesture_dmps[label]
    Y = dmp.rollout(T=T)          # (T, 32)
    skel_seq = vec32_to_skeleton(Y)  # (T, 16, 2)

    plot_stickman_frame2(ax, rotate_skeleton_xy(skel_seq), frame_idx=i)
    clear_output(wait=True)  # efface la sortie de la cellule
    display(fig)             # ré-affiche la figure
    plt.pause(0.05)          # petit délai pour voir l'update

  plt.close(fig)

In [43]:
# plot_stickman("one")
# plot_stickman("finger")
# plot_stickman("cup")
# plot_stickman("knock")
# plot_stickman("clasp")
# plot_stickman("hold")
# plot_stickman("punch")
# plot_stickman("point")
plot_stickman("wave")
# plot_stickman("show")
# plot_stickman("soft")
# plot_stickman("present")
# plot_stickman("hand")

KeyboardInterrupt: 